<a href="https://colab.research.google.com/github/genaiconference/Agentic_KAG/blob/main/05_Agentic_KAG_Inference_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic KAG


This notebook demonstrates the capabilities of Agentic KAG, a framework for building intelligent agents that can perform complex tasks and generate answers from Knoweldge Graphs.

In [ ]:
!git clone https://github.com/genaiconference/Agentic_KAG.git

fatal: destination path 'Agentic_KAG' already exists and is not an empty directory.


## Install Required Packages

In [ ]:
!pip install -r /content/Agentic_KAG/requirements.txt

## Load credentials from .env

In [ ]:
import os
os.chdir('/content/Agentic_KAG/')

from dotenv import load_dotenv

load_dotenv()  # This loads .env at project root

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set OPENAI_API_KEY as env variable for openai/neo4j-graphrag compatibility
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Initialize the Neo4j Driver
The Neo4j driver allows you to connect and perform read and write transactions with the database.

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# (Optional) Test the connection
driver.verify_connectivity()

## Initialize OpenAI LLM and Embeddings via neo4j-graphrag
We will use OpenAI **GPT-4.1**. The GraphRAG Python package supports any LLM model, including models from OpenAI, Google VertexAI, Anthropic, Cohere, Azure OpenAI, local Ollama models, and any chat model that works with LangChain. You can also implement a custom interface for any other LLM.

Likewise, we will use OpenAI’s **text-embedding-3-small** for the embedding model, but you can use other embedders from different providers.

In [ ]:
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4.1",
    model_params={"temperature": 0}
)

embedder = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

We will also initialize langchain llm to use for community detection & Summarization

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize OpenAI LLM using LangChain
lang_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4.1", temperature=0)

Initialize OpenAI client

In [ ]:
from openai import OpenAI

open_ai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

## Initialize Langfuse Handler for Tracing

In [ ]:
from langfuse.langchain import CallbackHandler
from langfuse import get_client

os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

langfuse_handler = CallbackHandler()

## Define ReACT Agent

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    PromptTemplate,
)

def get_react_agent(llm, tools, system_prompt, verbose=False):
    """Helper function for creating agent executor"""
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="conversation_history", optional=True),
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(input_variables=["input"], template="{input}")
            ),
            AIMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=["agent_scratchpad"], template="{agent_scratchpad}"
                )
            ),
        ]
    )
    agent = create_react_agent(llm, tools, prompt)
    return AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=verbose,
        stream_runnable=True,
        handle_parsing_errors=True,
        max_iterations=5,
        return_intermediate_steps=True,
    )

## Define Tools

Defining 4 tools used by the ReACT agent:
-  **Text2Cypher tool**
- **Hybrid Retrieval tool**
- **Global Search tool**
- **Local search tool**

In [ ]:
import tools

Got 14 community summaries


## Agentic KAG Pipeline

In [ ]:
from IPython.display import Markdown
import json
import prompts

def run_agentic_kag(query):
    """
    Runs the answer generation flow: Text2Cypher -> Hybrid Retrieval -> Final Answer.
    """
    # Run Hybrid Retrieval using all the available tools
    hybrid_agent = get_react_agent(
        lang_llm,
        [tools.av_hybrid_tool, tools.global_retriever_tool, tools.local_retriever_tool],
        prompts.REACT_PROMPT,
        verbose=True
    )
    hybrid_input = json.dumps({
        "query": query,
    })
    answer = hybrid_agent.invoke({"input": hybrid_input}, config={"callbacks": [langfuse_handler]})
    final_answer = answer['output']
    return final_answer

In [ ]:
query = "What are the main topics covered in the first half of Day 1."
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new AgentExecutor chain...
Thought: To answer this, I need to find the agenda or schedule for Day 1 and identify the main topics covered in the first half of the day.
Action: AVVectorRetrieval
Action Input: What are the main topics covered in the first half of Day 1?

> Entering new AgentExecutor chain...
Thought: I need to find sessions and workshops that occur on Day 1, in the first half (likely morning), and get the topics they cover. I should filter by sessions and workshops with start_time in the morning, and then get their related topics.
Action: text2cypher_tool
Action Input: What are the main topics covered by sessions and workshops that start on Day 1 in the morning?

CypherTypeError: {code: Neo.ClientError.Statement.TypeError} {message: Expected a string value for `toLower`, but got: StringArray[Day 1, Day 2]; consider converting it to a string with toString().}

In [ ]:
query = "List down all the sessions or workshops which uses LangGraph?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new AgentExecutor chain...
Thought: I need to find all sessions or workshops that mention the use of LangGraph.
Action: AVVectorRetrieval
Action Input: List all sessions or workshops that use LangGraph.

> Entering new AgentExecutor chain...
Thought: I need to find all Session and Workshop nodes that are connected to a Tool node with the name "LangGraph" via the USES relationship.
Action: text2cypher_tool
Action Input: List all sessions and workshops that use the tool named LangGraph.MATCH (s:Session)-[:USES]->(t:Tool)
WHERE toLower(t.name) CONTAINS toLower('LangGraph')
RETURN s.title AS title, s.description AS description, s.start_time AS start_time, s.end_time AS end_time, s.date AS date, s.venue AS venue, 'Session' AS type
UNION
MATCH (w:Workshop)-[:USES]->(t:Tool)
WHERE toLower(t.name) CONTAINS toLower('LangGraph')
RETURN w.title AS title, w.description AS description, w.start_time AS start_time, null AS end_time, null AS date, null AS venue, 'Workshop' AS typeFinal An

**

- **AutoGen vs CrewAI vs LangGraph: Battle of the Agent Frameworks**
  - Demonstrates and compares LangGraph alongside other frameworks.

- **Mastering Agentic Workflows with LangGraph**
  - Focuses specifically on building workflows with LangGraph.

- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Covers designing autonomous AI agents using LangGraph.

- **LLMOps – Productionalizing Real-World Applications with LLMs and Agents**
  - Includes orchestrating multi-agent workflows using LangGraph.

- **Building Intelligent Multimodal Agents: Integrating Vision, Speech & Language**
  - Teaches workflow creation and management with LangGraph.

- **Agentic RAG Workshop: From Fundamentals to Real-World Implementations**
  - Uses LangGraph for agent orchestration in hands-on modules.

- **Mastering Intelligent Agents: A Deep Dive into Agentic AI**
  - Leverages LangGraph (among other frameworks) for building Agentic AI systems.

In [ ]:
query = "I am a big lover of ai agents and I am interested in putting things in production. Suggest me a tailor made agenda, mention the name of the Session or workshop along with Instructor name?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new AgentExecutor chain...
Thought: The question is about creating a personalized agenda focused on AI agents and production deployment, including session/workshop names and instructor names. I need to search for relevant sessions and instructors.
Action: AVVectorRetrieval
Action Input: Sessions or workshops about AI agents and production deployment, including instructor names.

> Entering new AgentExecutor chain...
Thought: I need to find sessions or workshops that cover topics related to "AI agents" and "production deployment", and include instructor names for workshops.
Action: text2cypher_tool
Action Input: Find all Session or Workshop nodes where the title or description contains "AI agents" or "production deployment", and for workshops, return the instructor names.

CypherTypeError: {code: Neo.ClientError.Statement.TypeError} {message: Expected a string value for `toLower`, but got: StringArray[A high-stakes AI face-off as three leading multi-agent frameworks - AutoGen, CrewAI, and LangGraph, go head-to-head solving the same real-world AI problem: Building a Multi-Agent Helpdesk AI Assistant. Top Agentic AI practitioners demonstrate how each framework tackles this challenge: from structuring agent teams to orchestrating decisions across multiple steps. This unique session combines live hands-on demos and a panel discussion., Get ready for a high-stakes AI face-off as three leading multi-agent frameworks - AutoGen, CrewAI, and LangGraph, go head-to-head solving the same real-world AI problem: Building a Multi-Agent Helpdesk AI Assistant. Watch top Agentic AI practitioners demonstrate how each framework tackles this challenge: from structuring agent teams to orchestrating decisions across multiple steps. This unique session combines live hands-on de

In [ ]:
query = "I am a big lover of knowledge graphs (do not confuse with LangGraph). Suggest me a tailor made agenda, mention the name of the Session or workshop along with Instructor name?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new AgentExecutor chain...
Thought: The question is about creating a personalized agenda focused on knowledge graphs, specifically requesting session or workshop names and instructor names. I need to search for relevant sessions and instructors related to knowledge graphs.
Action: AVVectorRetrieval
Action Input: List all sessions or workshops related to knowledge graphs, including the session/workshop name and instructor name.

> Entering new AgentExecutor chain...
Thought: I need to find all Session or Workshop nodes that are related to the topic "knowledge graphs", and for each, return the session/workshop name and the instructor (for workshops) or presenter (for sessions) name.
Action: text2cypher_tool
Action Input: List all sessions or workshops that cover the topic "knowledge graphs", and return the session or workshop title and the name of the instructor or presenter.MATCH (s:Session)-[:COVERS]->(t:Topic)
WHERE toLower(t.name) CONTAINS toLower('knowledge graphs')
OPT

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: org.apache.lucene.queryparser.classic.TokenMgrError: Lexical error at line 1, column 117.  Encountered: <EOF> after prefix "/workshop name and instructor name." (in lexical state 2)}